In [1]:
#!/usr/bin/env python
import csv
import operator
import pprint
import sys
from copy import copy
import random as rn
from sklearn.cross_validation import KFold

#GREEDY SELECTION FNS BELOW
#------------------------------
#evalResults: given a dictionary of seqs mapped to predictions
#             determine results of the evaluation (FN, TN, FP, TP, error, accuracy etc)
def evalResults(dictofseqs,givendat,picked):
    falsepos = falseneg = truepos = trueneg = lowmiss = lowoccur = 0
    for x in range (1,len(givendat)):
        if(givendat[x][0] in dictofseqs):
            if(float(dictofseqs[givendat[x][0]]) == 1.0):
                if(float(givendat[x][len(givendat[0])-1]) == -1.0):
                    falsepos += 1
                elif(float(givendat[x][len(givendat[0])-1]) == 2.0):
                    lowoccur += 1
                else:
                    truepos += 1
                    if givendat[x][0] not in picked:
                        picked.append(givendat[x][0])
            else:
                if(float(givendat[x][len(givendat[0])-1]) == 1.0):
                    falseneg += 1
                elif(float(givendat[x][len(givendat[0])-1]) == 2.0):
                    lowmiss += 1
                else:
                    trueneg += 1
    tot = len(dictofseqs)
    overallerror = (falseneg+falsepos)/(trueneg+falseneg+falsepos+truepos)
    sens = truepos/(truepos+falseneg)
    spec = trueneg/(trueneg+falsepos)
    print("False Positives: " + str(falsepos))
    print("False Negatives: " + str(falseneg))
    print("True Positives: " + str(truepos))
    print("True Negatives: " + str(trueneg))
    print("CIMP-Low Occurrences: " + str(lowoccur))
    print("CIMP-Low Not Covered: " + str(lowmiss))
    print("Proportion of false positives: " + str((falsepos/tot)))
    print("Proportion of false negatives:" + str((falseneg/tot)))
    print("Accuracy: " + str((1-overallerror)))
    print("Error rate: " + str(overallerror))
    print("Sensitivity: " + str(sens))
    print("Specificity: " + str(spec))

def runGreedyOnTest(model, testdata):
    numrows = len(testdata)
    numcols = len(testdata[0])
    results = {}
    for x in range(1, numrows):
        for y in range(1, numcols-1): #program won't be able to see background/foreground
            if (float(testdata[x][y]) == 1.0):
                if (testdata[0][y] in model):
                    results[testdata[x][0]] = 1.0

    for x in range(0,numrows):
        if(testdata[x][0] not in results):
            results[testdata[x][0]] = -1.0
    return results

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#import data
with open("../../3ormoreDATLOW.csv",'r') as f:
    it = csv.reader(f)
    listit = list(it)

#set number of folds
numfolds = 10

#split data from header
header = listit[0]
listit = listit[1:]

#shuffle the data
rn.shuffle(listit)

#find testing index
kf = KFold(len(listit),n_folds=numfolds,random_state=True)
indices = []
for trainind,testind in kf:
    indices.append(testind)

In [3]:
for i in range(numfolds):
    #make testing and training sets
    trainset = [header]
    testset = [header]
    for x in range(0,len(listit)):
        if(x in indices[i]):
            testset.append(listit[x])
        else:
            trainset.append(listit[x])

    #setup pos-neg greedy
    perc = 0 #store the percent of foreground covered
    lenseq = len(trainset)
    seqnames = []
    F = []
    infoF = []
    SH = []
    SN = []
    negsamps = {}
    possamps = {}
    #get the negative and positive samples
    for x in range(1,lenseq):
        for y in range(0, len(trainset[x])-1):
            if y == 0:
                seqnames.append(trainset[x][y])
                continue
            if(listit[0][y] == ''):
                continue
            #initialize pos and neg
            if trainset[0][y] not in possamps:
                possamps[trainset[0][y]] = list()
            if trainset[0][y] not in negsamps:
                negsamps[trainset[0][y]] = list()
            if float(trainset[x][y]) == 1.0:
                if float(trainset[x][len(trainset[0])-1]) == 1.0:
                    possamps[trainset[0][y]].append(trainset[x][0])
                else:
                    negsamps[trainset[0][y]].append(trainset[x][0])

    while(True):
        maxscore = -sys.maxsize - 1
        sel = ""
        posexists = False
        #for each mutation, calculate a score
        for mut in possamps:
            if len(possamps[mut]) > 1:
                posexists = True #there still exist features which can bring in pos
            evaltot = (len(possamps[mut]) - len(negsamps[mut]))
            if evaltot > maxscore and len(possamps[mut]) > 1:
                maxscore = evaltot
                sel = mut
        if posexists == False:
            '''print("hello!",maxscore)
            print(possamps)'''
            break

        #calculate new percent covered
        count = len(possamps[sel])
        #print("max score: " + str(maxscore))
        #print("CIMP-H covered: " + str(count))
        perc += float(count/32) #add to percent covered
        myTup = (sel,perc)
        infoF.append(myTup)
        F.append(sel)
        for samp in possamps[sel]:
            SH.append(samp)
        for samp in negsamps[sel]:
            SN.append(samp)
        #print selected sequences (SF)
        #print(store[sel])
        posnew = {} #sequentially copy the new values into here
        negnew = {}
        for delseqs in possamps:
            if delseqs not in posnew:
                posnew[delseqs] = list()
                posnew[delseqs] = [y if y not in possamps[sel] else None for y in possamps[delseqs]]
                posnew[delseqs] = list(filter((None).__ne__, posnew[delseqs]))
        possamps = posnew
        possamps.pop(sel, None)
        for delseqs in negsamps:
            if delseqs not in negnew:
                negnew[delseqs] = list()
                negnew[delseqs] = [y if y not in negsamps[sel] else None for y in negsamps[delseqs]]
                negnew[delseqs] = list(filter((None).__ne__, negnew[delseqs]))

    foreback = {}
    
    '''#get pos/neg of samples for data purposes
    for x in range(1,lenseq):
        if(listit[x][0] in SN or listit[x][0] in SH):
            foreback[listit[x][0]] = listit[x][len(listit[0])-1]'''
    #F now contains the selected features
    print(infoF)
    print("-----------------------------------")
    #print(foreback)

    picked = []
    evalResults(runGreedyOnTest(F,testset),testset,picked)
    print("-----------------------------------")

[('BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T', 0.40625), ('MSH3_GRCh37_5:79970915-79970922_Frame-Shift-Del-DEL-AAAAAAAA----', 0.46875), ('ACVR2A_GRCh37_2:148683686-148683693_Frame-Shift-Del-DEL-AAAAAAAA----', 0.53125), ('KRAS_GRCh37_12:25398284-25398284_Missense-Mutation-SNP-C-T-T', 0.6875)]
-----------------------------------
False Positives: 1
False Negatives: 2
True Positives: 4
True Negatives: 16
CIMP-Low Occurrences: 0
CIMP-Low Not Covered: 0
Proportion of false positives: 0.041666666666666664
Proportion of false negatives:0.08333333333333333
Accuracy: 0.8695652173913043
Error rate: 0.13043478260869565
Sensitivity: 0.6666666666666666
Specificity: 0.9411764705882353
-----------------------------------
[('BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T', 0.46875), ('MBD4_GRCh37_3:129155548-129155557_Frame-Shift-Del-DEL-TTTTTTTTTT----', 0.5625), ('ACVR2A_GRCh37_2:148683686-148683693_Frame-Shift-